# Chapter 9 Evaluation (Part 1) - When there is a simple correct answer

- [I. Environment configuration](#I. Environment configuration)
- [1.1 Load the API key and some Python libraries. ](#1.1-Load-API-key-and-some-Python-libraries.)
- [1.2-Get-related-products-and-categories](#1.2-Get-related-products-and-categories)
- [II. Find-related-product-and-category-names](#II. Find-related-product-and-category-names)
- [III. Evaluate-on-some-queries](#III. Evaluate-on-some-queries)
- [IV. Harder-test-cases](#IV. Harder-test-cases)
- [V. Modify-instructions-to-handle-hard-test-cases](#V. Modify-instructions-to-handle-hard-test-cases)
- [VI. Evaluate-modified-instructions-on-hard-test-cases](#VI. Evaluate-modified-instructions-on-hard-test-cases)
- [VII. Regression-test: Verify-model-is-still-valid-on-previous-test-cases](#VII. Regression-test: Verify-model-is-still-valid-on-previous-test-cases)
- [VIII. Collect-development-sets-for-automated-tests](#VIII. Collect-development-sets-for-automated-tests)
- [Nine, evaluate test cases by comparing with ideal answers](#Nine, evaluate test cases by comparing with ideal answers)
- [Ten, run evaluation on all test cases and calculate the proportion of correct cases](#Ten,Run the evaluation on all test cases and calculate the proportion of correct cases)

In the previous chapters, we showed how to build applications using LLM, including evaluating inputs, processing inputs, and doing a final output check before showing the output to the user.

Once you build such a system, how do you know how well it works? Even after you deploy it and have users using it, how do you track how well it works, find any flaws, and continuously improve the quality of the system's answers?

In this chapter, we want to share with you some best practices for evaluating the output of LLM.

Building LLM-based applications is different from traditional supervised learning applications. Because LLM-based applications can be built quickly, the evaluation approach does not usually start with a test set. Instead, a set of test examples is usually built up gradually.

In traditional supervised learning settings, you need to collect a training set, a development set, or a holdout cross-validation set, and then use them throughout the development process.

However, if you can specify a prompt in a few minutes and get the corresponding results in a few hours, it would be extremely painful to pause for a long time to collect a thousand test examples. Because now, you can achieve this result with zero training examples.

So, when you build an application using LLM, you'll experience the following process:

First, you'll tweak the prompt on a small sample of one to three examples and try to get it to work on them.

Then, as the system is further tested, you might encounterSome tricky examples. Prompt doesn’t work on them, or the algorithm doesn’t work on them.

This is the challenge that developers who build applications using the ChatGPT API experience.

In this case, you can add these extra few examples to the set you are testing to opportunistically add other tricky examples.

Eventually, you have added enough of these examples to your slowly growing development set that it becomes somewhat inconvenient to test Prompt by manually running each example.

Then you start developing metrics for measuring performance on these small sets of examples, such as average accuracy.

An interesting aspect of this process is that if you feel your system is good enough, you can always stop there and not improve it anymore. In fact, many deployed applications stop at the first or second step and work very well.

It is important to note that there are many applications of large models that do not have substantial risk, even if it does not give completely correct answers.

However, for some high-risk applications, if there is bias or inappropriate output that could cause harm to someone, it becomes more important to collect a test set, rigorously evaluate the performance of the system, and ensure that it does the right thing before using it.

However, if you are only using it to summarize articles for your own reading, rather than for others, then the risk of harm is lower and you can stop early in the process rather than going to a more expensiveThe cost of collecting a larger dataset.

## 1. Environment Configuration

### 1.1 Load the API key and some Python libraries.

As in the previous chapter, we first need to configure the environment to use the OpenAI API

In [1]:
import os
import openai
import sys
import time
sys.path.append('../..')
import utils_en
import utils_zh

openai.api_key  = "sk-..."
# Set API_KEY, please replace it with your own API_KEY

# The following is an example of a configuration method based on environment variables, which is safer. It is for reference only and will not be covered later.
# import openai
# import os
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    封装一个访问 OpenAI GPT3.5 的函数

    参数: 
    messages: 这是一个消息列表，每个消息都是一个字典，包含 role(角色）和 content(内容)。角色可以是'system'、'user' 或 'assistant’，内容是角色的消息。
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 这决定模型输出的随机程度，默认为0，表示输出将非常确定。增加温度会使输出更随机。
    max_tokens: 这决定模型输出的最大的 token 数。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # 这决定模型输出的随机程度
        max_tokens=max_tokens, # 这决定模型输出的最大的 token 数
    )
    return response.choices[0].message["content"]

### 1.2 Get related products and categories

We want to get the list of products and categories in the product catalog mentioned in the previous chapters.

In [5]:
products_and_category = utils_en.get_products_and_category()
products_and_category

{'Computers and Laptops': ['TechPro Ultrabook',
  'BlueWave Gaming Laptop',
  'PowerLite Convertible',
  'TechPro Desktop',
  'BlueWave Chromebook'],
 'Smartphones and Accessories': ['SmartX ProPhone',
  'MobiTech PowerCase',
  'SmartX MiniPhone',
  'MobiTech Wireless Charger',
  'SmartX EarBuds'],
 'Televisions and Home Theater Systems': ['CineView 4K TV',
  'SoundMax Home Theater',
  'CineView 8K TV',
  'SoundMax Soundbar',
  'CineView OLED TV'],
 'Gaming Consoles and Accessories': ['GameSphere X',
  'ProGamer Controller',
  'GameSphere Y',
  'ProGamer Racing Wheel',
  'GameSphere VR Headset'],
 'Audio Equipment': ['AudioPhonic Noise-Canceling Headphones',
  'WaveSound Bluetooth Speaker',
  'AudioPhonic True Wireless Earbuds',
  'WaveSound Soundbar',
  'AudioPhonic Turntable'],
 'Cameras and Camcorders': ['FotoSnap DSLR Camera',
  'ActionCam 4K',
  'FotoSnap Mirrorless Camera',
  'ZoomMaster Camcorder',
  'FotoSnap Instant Camera']}

## 2. Find out related products and category names

In [4]:
def find_category_and_product_v1(user_input, products_and_category):
    """
    从用户输入中获取到产品和类别

    参数：
    user_input：用户的查询
    products_and_category：产品类型和对应产品的字典
    """

# Delimiter
    delimiter = "####"
# Defined system information, stating the work that needs to be done by GPT
    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be delimited with {delimiter} characters.
    Output a Python list of json objects, where each object has the following format:
        'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    AND
        'products': <a list of products that must be found in the allowed products below>


    Where the categories and products must be found in the customer service query.
    If a product is mentioned, it must be associated with the correct category in the allowed products list below.
    If no products or categories are found, output an empty list.
    

    List out all products that are relevant to the customer service query based on how closely it relates
    to the product name and product category.
    Do not assume, from the name of the product, any features or attributes such as relative quality or price.

    The allowed products are provided in JSON format.
    The keys of each item represent the category.
    The values of each item is a list of products that are within that category.
    Allowed products: {products_and_category}
    

    """
# Give a few examples
    few_shot_user_1 = """I want the most expensive computer."""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)


In [3]:
def find_category_and_product_v1(user_input,products_and_category):
    """
    从用户输入中获取到产品和类别

    参数：
    user_input：用户的查询
    products_and_category：产品类型和对应产品的字典
    """
    
    delimiter = "####"
    system_message = f"""
    您将提供客户服务查询。\
    客户服务查询将用{delimiter}字符分隔。
    输出一个 Python 列表，列表中的每个对象都是 Json 对象，每个对象的格式如下：
        'category': <Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders中的一个>,
    以及
        'products': <必须在下面允许的产品中找到的产品列表>
    
    其中类别和产品必须在客户服务查询中找到。
    如果提到了一个产品，它必须与下面允许的产品列表中的正确类别关联。
    如果没有找到产品或类别，输出一个空列表。
    
    根据产品名称和产品类别与客户服务查询的相关性，列出所有相关的产品。
    不要从产品的名称中假设任何特性或属性，如相对质量或价格。
    
    允许的产品以 JSON 格式提供。
    每个项目的键代表类别。
    每个项目的值是该类别中的产品列表。
    允许的产品：{products_and_category}
    
    """
    
    few_shot_user_1 = """我想要最贵的电脑。"""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)

## 3. Evaluate on some queries

In [5]:
# The first query to evaluate
customer_msg_0 = f"""Which TV can I buy if I'm on a budget?"""

products_by_category_0 = find_category_and_product_v1(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

    [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]


In [ ]:
# The first query to evaluate
customer_msg_0 = f"""如果我预算有限，我可以买哪款电视？"""

products_by_category_0 = find_category_and_product_v1(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

    [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'SoundMax Soundbar', 'CineView OLED TV']}]


In [6]:
# Second query to evaluate
customer_msg_1 = f"""I need a charger for my smartphone"""

products_by_category_1 = find_category_and_product_v1(customer_msg_1,
                                                      products_and_category)
print(products_by_category_1)

    [{'category': 'Smartphones and Accessories', 'products': ['MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds']}]



In [ ]:
customer_msg_1 = f"""我需要一个智能手机的充电器"""

products_by_category_1 = find_category_and_product_v1(customer_msg_1,
                                                      products_and_category)
print(products_by_category_1)

    [{'category': 'Smartphones and Accessories', 'products': ['MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds']}]



In [7]:
# Third evaluation query
customer_msg_2 = f"""
What computers do you have?"""

products_by_category_2 = find_category_and_product_v1(customer_msg_2,
                                                      products_and_category)
products_by_category_2

"    [{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]"

In [ ]:
customer_msg_2 = f"""
你们有哪些电脑？"""

products_by_category_2 = find_category_and_product_v1(customer_msg_2,
                                                      products_and_category)
products_by_category_2

"    [{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]"

In [10]:
# The fourth query is more complex
customer_msg_3 = f"""
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs do you have?"""

products_by_category_3 = find_category_and_product_v1(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

    [{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']},
     {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']},
     {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]
     
    Note: The query mentions "smartx pro phone" and "fotosnap camera, the dslr one", so the output includes the relevant categories and products. The query also asks about TVs, so the relevant category is included in the output.


In [ ]:
customer_msg_3 = f"""
告诉我关于smartx pro手机和fotosnap相机的信息，那款DSLR的。
另外，你们有哪些电视？"""

products_by_category_3 = find_category_and_product_v1(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

    [{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}]
    
    {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}


It looks like it outputs the correct data, but it also outputs a bunch of text, which is redundant. This makes it harder to parse it into a Python list of dictionaries.

## 4. More difficult test cases

Find some queries where the model performs worse than expected in actual use.

In [9]:
customer_msg_4 = f"""
tell me about the CineView TV, the 8K one, Gamesphere console, the X one.
I'm on a budget, what computers do you have?"""

products_by_category_4 = find_category_and_product_v1(customer_msg_4,
                                                      products_and_category)
print(products_by_category_4)

    [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 8K TV']},
     {'category': 'Gaming Consoles and Accessories', 'products': ['GameSphere X']},
     {'category': 'Computers and Laptops', 'products': ['BlueWave Chromebook']}]
     
    Note: The CineView TV mentioned is the 8K one, and the Gamesphere console mentioned is the X one. 
    For the computer category, since the customer mentioned being on a budget, we cannot determine which specific product to recommend. 
    Therefore, we have included all the products in the Computers and Laptops category in the output.


In [10]:
customer_msg_4 = f"""
告诉我关于CineView电视的信息，那款8K的，还有Gamesphere游戏机，X款的。
我预算有限，你们有哪些电脑？"""

products_by_category_4 = find_category_and_product_v1(customer_msg_4,products_and_category)
print(products_by_category_4)

    [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 8K TV']}, {'category': 'Gaming Consoles and Accessories', 'products': ['GameSphere X']}, {'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    
    具体来说，CineView 8K电视是一款高端电视，具有8K分辨率和OLED显示屏。GameSphere X是一款游戏机，具有高性能和多种游戏选择。对于预算有限的电脑，您可以考虑TechPro Chromebook或TechPro Ultrabook，它们都是较为经济实惠的选择。


## 5. Modify instructions to handle difficult test cases

We added the following to the prompt to not output any additional text that is not in JSON format, and added a second example for a few-shot prompt with user and assistant messages.

In [11]:
def find_category_and_product_v2(user_input, products_and_category):
    """
    从用户输入中获取到产品和类别
    添加：不要输出任何不符合 JSON 格式的额外文本。
    添加了第二个示例（用于 few-shot 提示），用户询问最便宜的计算机。
    在这两个 few-shot 示例中，显示的响应只是 JSON 格式的完整产品列表。

    参数：
    user_input：用户的查询
    products_and_category：产品类型和对应产品的字典
    """
    delimiter = "####"
    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be delimited with {delimiter} characters.
    Output a Python list of JSON objects, where each object has the following format:
        'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    AND
        'products': <a list of products that must be found in the allowed products below>
    Do not output any additional text that is not in JSON format.
    Do not write any explanatory text after outputting the requested JSON.


    Where the categories and products must be found in the customer service query.
    If a product is mentioned, it must be associated with the correct category in the allowed products list below.
    If no products or categories are found, output an empty list.
    

    List out all products that are relevant to the customer service query based on how closely it relates
    to the product name and product category.
    Do not assume, from the name of the product, any features or attributes such as relative quality or price.

    The allowed products are provided in JSON format.
    The keys of each item represent the category.
    The values of each item is a list of products that are within that category.
    Allowed products: {products_and_category}
    

    """
    
    few_shot_user_1 = """I want the most expensive computer. What do you recommend?"""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    few_shot_user_2 = """I want the most cheapest computer. What do you recommend?"""
    few_shot_assistant_2 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{few_shot_user_2}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_2 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)


In [11]:
def find_category_and_product_v2(user_input,products_and_category):
    """
    从用户输入中获取到产品和类别

    添加：不要输出任何不符合 JSON 格式的额外文本。
    添加了第二个示例（用于 few-shot 提示），用户询问最便宜的计算机。
    在这两个 few-shot 示例中，显示的响应只是 JSON 格式的完整产品列表。

    参数：
    user_input：用户的查询
    products_and_category：产品类型和对应产品的字典    
    """
    delimiter = "####"
    system_message = f"""
    您将提供客户服务查询。\
    客户服务查询将用{delimiter}字符分隔。
    输出一个 Python列表，列表中的每个对象都是 JSON 对象，每个对象的格式如下：
        'category': <Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders中的一个>,
    AND
        'products': <必须在下面允许的产品中找到的产品列表>
    不要输出任何不是 JSON 格式的额外文本。
    输出请求的 JSON 后，不要写任何解释性的文本。
    
    其中类别和产品必须在客户服务查询中找到。
    如果提到了一个产品，它必须与下面允许的产品列表中的正确类别关联。
    如果没有找到产品或类别，输出一个空列表。
    
    根据产品名称和产品类别与客户服务查询的相关性，列出所有相关的产品。
    不要从产品的名称中假设任何特性或属性，如相对质量或价格。
    
    允许的产品以 JSON 格式提供。
    每个项目的键代表类别。
    每个项目的值是该类别中的产品列表。
    允许的产品：{products_and_category}
    
    """
    
    few_shot_user_1 = """我想要最贵的电脑。你推荐哪款？"""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    few_shot_user_2 = """我想要最便宜的电脑。你推荐哪款？"""
    few_shot_assistant_2 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{few_shot_user_2}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_2 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)

## 6. Evaluating the modified instructions on difficult test cases

In [12]:
customer_msg_3 = f"""
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs do you have?"""

products_by_category_3 = find_category_and_product_v2(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

    [{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]



In [12]:
customer_msg_3 = f"""
告诉我关于smartx pro手机和fotosnap相机的信息，那款DSLR的。
另外，你们有哪些电视？"""

products_by_category_3 = find_category_and_product_v2(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

    [{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]



## VII. Regression testing: Verify that the model is still valid on previous test cases

Check and fix the model to improve the performance of difficult-to-test cases, while ensuring that this correction does not negatively affect the performance of previous test cases.

In [13]:
customer_msg_0 = f"""Which TV can I buy if I'm on a budget?"""

products_by_category_0 = find_category_and_product_v2(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

    [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]



In [13]:
customer_msg_0 = f"""如果我预算有限，我可以买哪款电视？"""

products_by_category_0 = find_category_and_product_v2(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

 

    [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]
    
    如果您的预算有限，我们建议您购买CineView 4K电视或SoundMax家庭影院。


## 8. Collect development sets for automated testing

When you have a development set that you want to tune that is more than just a small set of examples, it becomes useful to start automating your testing process.

In [14]:
msg_ideal_pairs_set = [
    
# eg 0
    {'customer_msg':"""Which TV can I buy if I'm on a budget?""",
     'ideal_answer':{
        'Televisions and Home Theater Systems':set(
            ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']
        )}
    },

# eg 1
    {'customer_msg':"""I need a charger for my smartphone""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds']
        )}
    },
# eg 2
    {'customer_msg':f"""What computers do you have?""",
     'ideal_answer':{
           'Computers and Laptops':set(
               ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'
               ])
                }
    },

# eg 3
    {'customer_msg':f"""tell me about the smartx pro phone and \
    the fotosnap camera, the dslr one.\
    Also, what TVs do you have?""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['SmartX ProPhone']),
        'Cameras and Camcorders':set(
            ['FotoSnap DSLR Camera']),
        'Televisions and Home Theater Systems':set(
            ['CineView 4K TV', 'SoundMax Home Theater','CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV'])
        }
    }, 
    
# eg 4
    {'customer_msg':"""tell me about the CineView TV, the 8K one, Gamesphere console, the X one.
I'm on a budget, what computers do you have?""",
     'ideal_answer':{
        'Televisions and Home Theater Systems':set(
            ['CineView 8K TV']),
        'Gaming Consoles and Accessories':set(
            ['GameSphere X']),
        'Computers and Laptops':set(
            ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'])
        }
    },
    
# eg 5
    {'customer_msg':f"""What smartphones do you have?""",
     'ideal_answer':{
           'Smartphones and Accessories':set(
               ['SmartX ProPhone', 'MobiTech PowerCase', 'SmartX MiniPhone', 'MobiTech Wireless Charger', 'SmartX EarBuds'
               ])
                    }
    },
# eg 6
    {'customer_msg':f"""I'm on a budget.  Can you recommend some smartphones to me?""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['SmartX EarBuds', 'SmartX MiniPhone', 'MobiTech PowerCase', 'SmartX ProPhone', 'MobiTech Wireless Charger']
        )}
    },

# eg 7 # this will output a subset of the ideal answer
    {'customer_msg':f"""What Gaming consoles would be good for my friend who is into racing games?""",
     'ideal_answer':{
        'Gaming Consoles and Accessories':set([
            'GameSphere X',
            'ProGamer Controller',
            'GameSphere Y',
            'ProGamer Racing Wheel',
            'GameSphere VR Headset'
     ])}
    },
# eg 8
    {'customer_msg':f"""What could be a good present for my videographer friend?""",
     'ideal_answer': {
        'Cameras and Camcorders':set([
        'FotoSnap DSLR Camera', 'ActionCam 4K', 'FotoSnap Mirrorless Camera', 'ZoomMaster Camcorder', 'FotoSnap Instant Camera'
        ])}
    },
    
# eg 9
    {'customer_msg':f"""I would like a hot tub time machine.""",
     'ideal_answer': []
    }
    
]


## 9. Evaluate test cases by comparing with ideal answers

In [16]:
import json
def eval_response_with_ideal(response,
                              ideal,
                              debug=False):
    """
    评估回复是否与理想答案匹配
    
    参数：
    response: 回复的内容
    ideal: 理想的答案
    debug: 是否打印调试信息
    """
    if debug:
        print("回复：")
        print(response)
    
# json.loads() can only parse double quotes, so replace single quotes with double quotes here
    json_like_str = response.replace("'",'"')
    
# Parse into a series of dictionaries
    l_of_d = json.loads(json_like_str)
    
# When the response is empty, that is, no products are found
    if l_of_d == [] and ideal == []:
        return 1
    
# Another abnormal situation is that the number of standard answers does not match the number of reply answers
    elif l_of_d == [] or ideal == []:
        return 0
    
# Count the number of correct answers
    correct = 0    
    
    if debug:
        print("l_of_d is")
        print(l_of_d)

# For each question and answer pair
    for d in l_of_d:

# Get products and categories
        cat = d.get('category')
        prod_l = d.get('products')
# Products and catalogs are obtained
        if cat and prod_l:
# convert list to set for comparison
            prod_set = set(prod_l)
# get ideal set of products
            ideal_cat = ideal.get(cat)
            if ideal_cat:
                prod_set_ideal = set(ideal.get(cat))
            else:
                if debug:
                    print(f"没有在标准答案中找到目录 {cat}")
                    print(f"标准答案: {ideal}")
                continue
                
            if debug:
                print("产品集合：\n",prod_set)
                print()
                print("标准答案的产品集合：\n",prod_set_ideal)

# The found product set is consistent with the standard product set
            if prod_set == prod_set_ideal:
                if debug:
                    print("正确")
                correct +=1
            else:
                print("错误")
                print(f"产品集合: {prod_set}")
                print(f"标准的产品集合: {prod_set_ideal}")
                if prod_set <= prod_set_ideal:
                    print("回答是标准答案的一个子集")
                elif prod_set >= prod_set_ideal:
                    print("回答是标准答案的一个超集")

# Count the number of correct answers
    pc_correct = correct / len(l_of_d)
        
    return pc_correct

In [16]:
print(f'用户提问: {msg_ideal_pairs_set[7]["customer_msg"]}')
print(f'标准答案: {msg_ideal_pairs_set[7]["ideal_answer"]}')

用户提问: What Gaming consoles would be good for my friend who is into racing games?
标准答案: {'Gaming Consoles and Accessories': {'GameSphere VR Headset', 'GameSphere X', 'ProGamer Controller', 'ProGamer Racing Wheel', 'GameSphere Y'}}


In [17]:
response = find_category_and_product_v2(msg_ideal_pairs_set[7]["customer_msg"],
                                         products_and_category)
print(f'回答: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[7]["ideal_answer"])

回答:     [{'category': 'Gaming Consoles and Accessories', 'products': ['ProGamer Controller', 'ProGamer Racing Wheel', 'GameSphere VR Headset']}]
错误
产品集合: {'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere VR Headset'}
标准的产品集合: {'GameSphere VR Headset', 'GameSphere X', 'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere Y'}
回答是标准答案的一个子集


0.0

In [17]:
# Call Chinese Prompt
response = find_category_and_product_v2(msg_ideal_pairs_set[7]["customer_msg"],
                                         products_and_category)
print(f'回答: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[7]["ideal_answer"])

回答:     [{'category': 'Gaming Consoles and Accessories', 'products': ['GameSphere X', 'ProGamer Controller', 'GameSphere Y', 'ProGamer Racing Wheel', 'GameSphere VR Headset']}]


0.0

## 10. Run the evaluation on all test cases and calculate the correct use case ratio

Note: If any API call times out, it will not run

In [20]:
score_accum = 0
for i, pair in enumerate(msg_ideal_pairs_set):
    time.sleep(20)
    print(f"示例 {i}")
    
    customer_msg = pair['customer_msg']
    ideal = pair['ideal_answer']
    
# print("Customer message",customer_msg)
# print("ideal:",ideal)
    response = find_category_and_product_v2(customer_msg,
                                                      products_and_category)

    
# print("products_by_category",products_by_category)
    score = eval_response_with_ideal(response,ideal,debug=False)
    print(f"{i}: {score}")
    score_accum += score
    

n_examples = len(msg_ideal_pairs_set)
fraction_correct = score_accum / n_examples
print(f"正确比例为 {n_examples}: {fraction_correct}")

示例 0
0: 1.0
示例 1
1: 1.0
示例 2
2: 1.0
示例 3
3: 1.0
示例 4
4: 1.0
示例 5
5: 1.0
示例 6
6: 1.0
示例 7
错误
产品集合: {'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere VR Headset'}
标准的产品集合: {'GameSphere VR Headset', 'GameSphere X', 'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere Y'}
回答是标准答案的一个子集
7: 0.0
示例 8
8: 1.0
示例 9
9: 1
正确比例为 10: 0.9


The workflow for building an application with Prompt is very different from the workflow for building an application with supervised learning.

So we think that's a good thing to keep in mind, and it feels like you can iterate much faster when you're building supervised learning models.

If you haven't seen it yourself, you might be surprised at how few examples you can manually construct to produce an effective evaluation method. You might think that just 10 examples is not statistically significant. But when you really use it, you might be surprised at the improvement you get by adding some complex examples to your development set.

This is very helpful in helping you and your team find effective prompts and effective systems.

In this chapter, the output can be evaluated quantitatively, just like there is an expected output, and you can tell whether it gave that expected output.

In the next chapter, we'll explore how to evaluate our output in more ambiguous situations. That is, situations where the correct answer may not be so clear.